In [1]:
import wikipedia as wp
import requests
import bs4
import os

In [2]:
files = {}

with open('languages.txt', 'r') as file:
    languages = [f.strip('\n') for f in file.readlines()]

In [3]:
headers = {'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/77.0.3865.90 Safari/537.36'}

# FIND 100 MOST INFLUENTIAL PEOPLE

In [4]:
# url of 100 most influential people in history
url = 'https://www.biographyonline.net/people/100-most-influential.html'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, 'html5lib')

## EXTRACT NAMES

In [5]:
name_elements = soup.find_all('li', class_ = None)[:100]

In [6]:
def get_name_from_element(tag):
    try:
        return tag.find_all('a')[0].text.strip()
    except:
        ret = tag.text.split('(')[0].strip()
        if 'Menes' in ret:
            return ret.split(' ')[0]
        else:
            return ret

In [7]:
names = [get_name_from_element(elem) for elem in name_elements]

## EXTRACT CONTENT OF THEIR WIKIPEDIA PAGES

In [8]:
# create dir if not exists
if not os.path.exists('page_text'):
    os.makedirs('page_text') 

# download pages in every languages
for lang in languages:
    print(lang)
    wp.set_lang(lang)
    files[lang] = {}
    for i, name in enumerate(names):
        if i % 10 == 0:
            print(i)
        try:
            page_content = wp.page(wp.search(name)[0]).content
            files[lang][name] = page_content
        except:
            continue

        

en
0
10
20
30
40
50
60
70
80


/home/kapronczaym/.local/lib/python3.6/site-packages/bs4/__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file /usr/lib/python3.6/runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup([your markup])

to this:

 BeautifulSoup([your markup], "lxml")

  markup_type=markup_type))


90
de
0
10
20
30
40
50
60
70
80
90
hu
0
10
20
30
40
50
60
70
80
90
ro
0
10
20
30
40
50
60
70
80
90


# FIND 100 BEST CITIES

In [9]:
# url of 100 best cities to live list
url = 'https://www.bestcities.org/rankings/worlds-best-cities/'
r = requests.get(url)
soup = bs4.BeautifulSoup(r.content, 'html5lib')

In [10]:
cities = [' '.join(match.text.strip().split(' ')[1:]) for match in soup.findAll('div', {'class': 'rankings-cities-detail'})]

## FIND THEIR WIKIPEDIA PAGES

#### EXTRACT CONTENT

In [11]:
# download pages in every language
for lang in languages:
    print(lang)
    wp.set_lang(lang)
    for i, name in enumerate(cities):
        if i % 10 == 0:
            print(i)
        try:
            page_content = wp.page(wp.search(name)[0]).content
            files[lang][name] = page_content
        except:
            continue


en
0
10
20
30
40
50
60
70
80
90
de
0
10
20
30
40
50
60
70
80
90
hu
0
10
20
30
40
50
60
70
80
90
ro
0
10
20
30
40
50
60
70
80
90


# FIND TOP 100 COMPANIES

In [12]:
# url of top 100 companies list
url = 'https://ceoworld.biz/2019/06/28/the-top-100-best-performing-companies-in-the-world-2019/'
r = requests.get(url, headers=headers)
soup = bs4.BeautifulSoup(r.content, 'html5lib')

In [13]:
companies = [f.text for f in soup.findAll('td', {'class': 'column-2'})]

## FIND THEIR WIKIPEDIA PAGES

#### EXTRACT CONTENT

In [14]:
# download pages in every language
for lang in languages:
    print(lang)
    wp.set_lang(lang)
    for i, name in enumerate(companies):
        if i % 10 == 0:
            print(i)
        try:
            page_content = wp.page(wp.search(name)[0]).content
            files[lang][name] = page_content
        except:
            continue

en
0
10
20
30
40
50
60
70
80
90
de
0
10
20
30
40
50
60
70
80
90
hu
0
10
20
30
40
50
60
70
80
90
ro
0
10
20
30
40
50
60
70
80
90


# FIND TOP100 POP/ROCK BANDS

In [15]:
# url of top 100 companies list
url = 'https://www.imdb.com/list/ls076954447/'
r = requests.get(url, headers=headers)
soup = bs4.BeautifulSoup(r.content, 'html5lib')

In [16]:
bands = [f.text.strip().split('\n')[-1].strip() for f in soup.findAll('h3', {'class': 'lister-item-header'})]

## FIND THEIR WIKIPEDIA PAGES

#### EXTRACT CONTENT

In [17]:
# download pages in every language
for lang in languages:
    print(lang)
    wp.set_lang(lang)
    for i, name in enumerate(bands):
        if i % 10 == 0:
            print(i)
        try:
            page_content = wp.page(wp.search(name)[0]).content
            files[lang][name] = page_content
        except:
            continue

en
0
10
20
30
40
50
60
70
80
90
de
0
10
20
30
40
50
60
70
80
90
hu
0
10
20
30
40
50
60
70
80
90
ro
0
10
20
30
40
50
60
70
80
90


#### KEEP ONLY PAGES WHICH ARE PRESENT IN ALL LANGUAGES

In [18]:
for i, key in enumerate(files.keys()):
    if i == 0:
        common_files = set(files[key])
    else:
        common_files = common_files.intersection(set(files[key]))

In [19]:
len(common_files)

328

#### WRITE TO FILE

In [22]:
for lang in languages:
    if not os.path.exists(os.path.join('page_text', f'{lang}')):
        os.makedirs(os.path.join('page_text', f'{lang}'))
    for doc in common_files:
        with open((os.path.join('page_text', f'{lang}', f'{doc.lower().replace(" ", "-").replace("/", "")}.txt')), 'w') as file:
            file.write(files[lang][doc])